In [ ]:
import sys

sys.path.insert(1, "../")

import glob
from dem_comparison.plots import plot_metrics
from pathlib import Path
from dem_comparison.utils import resample_dataset
import multiprocessing as mp

In [ ]:
is_error = True
polar = True
pkls = glob.glob(f"../Antarctica_Dem_Comparison/dem_diff_metrics/**.pkl")
plot_metrics(
    pkls,
    is_error,
    polar,
    "plot.html",
    data_bounds=[(-50, 50), (0, 10), (0, 250), (0, 5)],
    num_bins=15,
)

In [ ]:
overviews_path = Path("../Antarctica_Dem_Comparison/overviews")
overviews_path.mkdir(parents=True, exist_ok=True)
mosaic_files = [
    Path(f) for f in glob.glob("../Antarctica_Dem_Comparison/mosaics/*.tif")
]
with mp.Pool(processes=6) as p:
    try:
        p.starmap(
            resample_dataset,
            [
                (
                    file,
                    0.2,
                    str(overviews_path / file.name),
                )
                for file in mosaic_files
            ],
        )
    except Exception as e:
        raise e

In [ ]:
from dem_comparison.utils import simple_mosaic

tifs = glob.glob("../Antarctica_Dem_Comparison/overviews/*.tif")
simple_mosaic(tifs, "../Antarctica_Dem_Comparison/mosaic_downsampled.tif")